<a href="https://colab.research.google.com/github/Rchddeg/Coursera_Capstone/blob/master/Capstone_project_The_Battle_of_Neighbourhoods_Restaurant_in_Rdam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Description of the Problem



In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an Italian restaurant in Rotterdam, The Netherlands.

Description of the Data

ased on definition of our problem, factors that will influence our decission are:

- number of existing restaurants in the neighborhood (any type of restaurant)
- number of and distance to Italian restaurants in the neighborhood, if any 
- distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:

- centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Google Maps API reverse geocoding
- number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API
- coordinates of Rotterdam center will be obtained using Google Maps API geocoding of well known Rotterdam location (Coolsingel)



**Neighborhood Candidates** 

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 6x6 killometers centered around Rotterdam city center.

Let's first find the latitude & longitude of Rotterdam city center, using specific, well known address and Geocoder with arcGIS API.



In [0]:
#!pip install geocoder
import geocoder

def get_coordinates(address):
  g = geocoder.arcgis(address)
  return g.latlng

address = "Coolsingel, Rotterdam, NL"
rdam_center = get_coordinates(address)
#print('Coordinate of {}: {}').format(address, rdam_center))
print('Coordinate of {}: {}'.format(address, rdam_center))

Coordinate of Coolsingel, Rotterdam, NL: [51.921009741358645, 4.47966432595502]


let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Coolsingel. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then those coordinates will be projected back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [0]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Rotterdam center longitude={}, latitude={}'.format(rdam_center[1], rdam_center[0]))
x, y = lonlat_to_xy(rdam_center[1], rdam_center[0])
print('Rotterdam center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Rotterdam center longitude={}, latitude={}'.format(lo, la))

     |████████████████████████████████| 10.4MB 3.1MB/s 
Coordinate transformation check
-------------------------------
Rotterdam center longitude=4.47966432595502, latitude=51.921009741358645
Rotterdam center UTM X=-222506.84024270345, Y=5804726.6885958845
Rotterdam center longitude=4.479664325955022, latitude=51.921009741358645




Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.


In [0]:
# City center in Cartesian coordinates
rdam_center_x, rdam_center_y = lonlat_to_xy(rdam_center[1], rdam_center[0]) 

# Vertical offset for hexagonal grid cells
k = math.sqrt(3) / 2 
x_min = rdam_center_x - 6000
x_step = 600
y_min = rdam_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(rdam_center_x, rdam_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's showcase what we have on a folium map:
The city center location and candidate neighborhood centers



In [0]:
import folium

map_rdam = folium.Map(location=rdam_center, zoom_start=13)
folium.Marker(rdam_center, popup='Coolsingel').add_to(map_rdam)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='red', fill=True, fill_color='green', fill_opacity=1).add_to(map_rdam) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_rdam)
    #folium.Marker([lat, lon]).add_to(map_rdam)
map_rdam

Now we have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~ 6km from Coolsingel. Let's now use Geocoder to get approximate addresses of those locations.

In [0]:
##CONTINUE HERE WITH BING + API KEY
key = 'AjPYTSlYCGj1MxECPvPeeVAG6wXNRUyOs4xTvkFSb7vk8OvAPThy4B9j_RFRBlBt'

addresses = []
latlng = [51.921009741358645, 4.47966432595502]
g = geocoder.bing(latlng, method='reverse', key=key)

print(g)

addresses.append(g)
print(addresses)


<[OK] Bing - Reverse [Coolsingel 48B, 3011 AD Rotterdam, Netherlands]>
[<[OK] Bing - Reverse [Coolsingel 48B, 3011 AD Rotterdam, Netherlands]>]
